<!-- ## SMALL-SCALE CROP-MAPPING(Food security) -->

<center><h1>BIOMASS APPLICATION</h1> </center>

In [1]:
#Importation of variAous Libraries or packages...
import os
from os import path as op
import geopandas as gdp
import geemap
import ee


# Making module or packages collection attribute became callable..
import collections
collections.Callable = collections.abc.Callable

#plotting packages
import pandas as pd#Used for data analysis also include DataFrame df data structure
import matplotlib.pyplot as plt#For plotting of dataframes
import numpy as np
import altair as alt#declarative visualization(Used for charting)

import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl
from ipywidgets import HTML
from IPython.display import display, clear_output
from ipywidgets import AppLayout

from geemap import chart #for chart plotting


#packages used in database connection.
import pandas.io.sql as sqlio
import psycopg2 as ps

#Main function is to out various warning that might arise such as new upadates in package version
import warnings
warnings.filterwarnings("ignore")
Map=geemap.Map()
from geemap import Map
from ipywidgets import Button, Layout

map_layout= Map(layout=Layout(height='100px'))
map_layout.setCenter(76.3213, 29.8419,17)
 
map_layout.add_basemap(basemap='HYBRID')

In [2]:
#Setting various buttons for the final dsplay of the results and the output widgets or setting the intractive keys
style = {'description_width': 'initial'}
#General output widget
output_widget = widgets.Output(layout={'border': '1px solid red','align-items':'center','background-color':'red'})

#Widget used to display crop health
output_widget_health = widgets.Output(layout={'border': '1px solid black','marginright':'150px'})

#General output control tool.
output_control = WidgetControl(widget=output_widget, position='topright')
#Map output control
map_layout.add_control(output_control)

#BUTTON  and TEXT SECTION 
#styling method that can be used-----'primary', 'success', 'info', 'warning', 'danger',)

#AOI Area of Interest Selection of study area.
AOI = widgets.Button(
    description='Drawn Study Area',
    button_style='success',
    tooltip='Draw you boundaries',
    style=style
)


Image_Preprocessing = widgets.Button(
    description='Image Preprocessing',
    button_style='info',
    tooltip='Image Preprocessing process',
    style=style
)
Update_Mask = widgets.Button(
    description='Mask Cropland',
    button_style='danger',
    tooltip='Mask Cropland',
    style=style
)

load_model = widgets.Button(
    description=' Image Classification',
    button_style='primary',
    tooltip='Load the machine learned model',
    style=style
)

Export_data = widgets.Button(
    description='Calculate Mean',
    button_style='success',
    tooltip='Generate Mean Value',
    style=style
)
GPP_EXECUTE = widgets.Button(
    description='Calculate GPP',
    button_style='warning',
    tooltip='Gross primary product calculatation',
    style=style
)

Reset_button= widgets.Button(
    description='Reset',
    button_style='danger',
    tooltip='reset by use of this button',
    style=style
)
Data_Input = widgets.Button(
    description='Estimate Biomass Value',
    button_style='info',
    tooltip='Estimate Biomass Value',
    style=style
)

NDVI_execute = widgets.Button(
    description='Calculate NDVI',
    button_style='primary',
    tooltip='Calculate NDVI',
    style=style
)

LULCC = widgets.Button(
    description='Dynamic LULC',
    button_style='info',
    tooltip='Dynamic Worlds',
    style=style
)


In [3]:
output_widget88 = widgets.Output(layout={'border': '1px solid black'})
output_control88 = WidgetControl(widget=output_widget88, position='bottomright')
map_layout.add_control(output_control88)

In [4]:
style = {'description_width': 'initial'}
uploader = widgets.FileUpload(
    description='Upload data',
    accept='.zip, .json, .geojson',
    multiple=False,
    button_style='primary',
    style=style,
)

submit = widgets.Button(
    description='Display data', button_style='success', tooltip='Click me', style=style
)

reset = widgets.Button(
    description='Reset', button_style='warning', tooltip='Click me', style=style
)

In [5]:
with output_widget88:
    print('Upload shapefile or \ngeojson as a zip file')
    display(uploader)
    display(submit)
    display(reset)

In [6]:
def get_vector(upload_widget, out_dir=None):

    import zipfile
    import glob

    if out_dir is None:
        out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    vector = None

    try:
        [uploaded_file] = upload_widget.value
        file = upload_widget.value[uploaded_file]
        name = file['metadata']['name']
        content = file['content']
        out_file = os.path.join(out_dir, name)
        with open(out_file, "wb") as fp:
            fp.write(content)

        if name.endswith('.zip'):
            with zipfile.ZipFile(out_file, "r") as zip_ref:
                extract_dir = os.path.join(
                    out_dir, name[:-4] + "_" + geemap.random_string(3)
                )
                zip_ref.extractall(extract_dir)
                files = glob.glob(extract_dir + '/*.shp')
                if len(files) > 0:
                    shp = files[0]
                    vector = geemap.shp_to_ee(shp)
                else:
                    files = glob.glob(extract_dir + '/*.geojson')
                    if len(files) > 0:
                        geojson = files[0]
                        vector = geemap.geojson_to_ee(geojson)
        else:
            vector = geemap.geojson_to_ee(out_file)

    except Exception as e:
        print(e)

    return vector

In [7]:
def submit_clicked(b):
    if uploader._counter > 0:
        map_layout.default_style = {'cursor': 'wait'}
        try:
            fc = get_vector(uploader)
            layer_name = 'Layer ' + geemap.random_string(3)
            map_layout.addLayer(fc, {}, layer_name)
            map_layout.centerObject(fc)
            uploader.value.clear()
            uploader._counter = 0
        except Exception as e:
            print(e)
        map_layout.default_style = {'cursor': 'pointer'}


submit.on_click(submit_clicked)


def reset_clicked(b):

    map_layout.layers = map_layout.layers[:3]
    output_widget88.clear_output()
    with output_widget88:
        print('Upload shapefile or \ngeojson as a zip file')
        display(uploader)
        display(submit)
        display(reset)
    uploader.value.clear()
    uploader._counter = 0


reset.on_click(reset_clicked)

In [8]:
#User instructions manual.
instructions=widgets.Dropdown(
    options=['Please follow this steps sequentially',
             '1.Drawn study area or upload',
             '2.Carry out image pre-processing',
             '3.Image classification(Only applicable if you have the points,skip if not AOI)',
             '4.Alternatively use Dynamic World for image classification(if you want to work with another study area)',
             '5.Calculate Gross Primary Product',
             '6.Extract the GPP and mean value',
             '7.Calculate NDVI Readings',
             '8.Estimate Biomass Readings',
             '9.Reset or clear everything'],
    value='Please follow this steps sequentially',
    description='Instruction:',
    disabled=False,
)

In [9]:
#Arrange the layout
from ipywidgets import AppLayout
verticalBox = output_widget
vBox1 = widgets.VBox([map_layout])


vBox2 = widgets.VBox([instructions,AOI, Image_Preprocessing,load_model,LULCC,Update_Mask,GPP_EXECUTE,NDVI_execute,Export_data,Data_Input,Reset_button])

AppLayout(header=None,
          left_sidebar=vBox1,
          right_sidebar=vBox2,
          footer=None,
          step= 1,
          pane_widths=[3, 3, 1],
          pane_heights=[1, 1, '100%'])

AppLayout(children=(VBox(children=(Map(center=[29.8419, 76.3213], controls=(WidgetControl(options=['position',…

In [10]:
#Trial to avoid printing error in the code but a message to the user
def AOI_clicked(b):
        with output_widget:
#             output_widget.clear_output()
           
            try:
                global boundaries
                boundaries=map_layout.user_rois
                map_layout.addLayer(boundaries,{},"Region of Intrest")
            
         
            except Exception as e:
                print('Please select Your area of Interest.....')
            else:
                print("Successfully drawn your Region of Interest ")
                output_widget.clear_output()
        
AOI.on_click(AOI_clicked)

In [11]:
#Trial to avoid printing error in the code but a message to the user
def AOI_clicked(b):
        with output_widget:
            map_layout.remove_legends() 
            map_layout.remove_colorbars()
            
            global npp
            npp=ee.ImageCollection("MODIS/006/MOD17A3H")
            global gpp
            gpp=ee.ImageCollection("MODIS/006/MYD17A2H")
            map_layout.addLayer(boundaries,{},"boundaries")
            map_layout.centerObject(boundaries,17)
            global startdate
            startdate = ee.Date.fromYMD(2006,1,1)
            global enddate
            enddate = ee.Date.fromYMD(2012,12,31)
      
            startdate_GPP = ee.Date.fromYMD(2006,1,1)
           
            enddate_GPP = ee.Date.fromYMD(2021,12,31)
            global gppCollection
            nppCollection = npp.filterDate(startdate, enddate) \
           
            gppCollection = gpp.filterDate(startdate_GPP, enddate_GPP) \
                       .select("Gpp")
            def myNpp(myimg):
                d = ee.Date(myimg.get('system:time_start'))
                y = d.get('year').toInt()
                GPPy = ee.Image(gppCollection.filter(ee.Filter.calendarRange(y, y, 'year')).sum())
                NPPy = ee.Image(nppCollection.filter(ee.Filter.calendarRange(y, y, 'year')).mean())
                npp8 = myimg.expression('(GGP8 / GPPy) * NPPy',
                                        {
        'GGP8': myimg,
        'GPPy': GPPy,
        'NPPy': NPPy
    })
                out1 = npp8.copyProperties(myimg,['system:time_start'])
                return out1
            npp8Collection = ee.ImageCollection(gppCollection.map(myNpp))
            
            def Biomass(myimg):
                
                biomass = myimg.multiply(2.5)
                return biomass.copyProperties(myimg,['system:time_start'])
            
            biomassCollection = npp8Collection.map(Biomass)
            
         
            band_viz = {
  'min': 0,
  'max': 600,
  'palette': ['bbe029', '0a9501', '074b03']
}
       
            map_layout.addLayer(gppCollection.mean().clip(boundaries),band_viz,"Gross Primary Product")
            vis_params = {
                    'min': 0,
                    'max': 600,
                    'palette':['bbe029', '0a9501', '074b03'],
                }
            colors = vis_params['palette']
            vmin = vis_params['min']
            vmax = vis_params['max']
            map_layout.add_colorbar(vis_params,label='Gross Primary Product(GPP)')
        
GPP_EXECUTE.on_click(AOI_clicked)

In [12]:
# Click event handler for Image processing process e.g sentinel data

def preprocess_img_clicked(b):
     with output_widget:
            output_widget.clear_output()
            try:
                start_date="2020-01-01"#Set start_date(yy/mon/day)
                end_date="2020-03-31"#Set End_date(yy/mon/day)
                season = ee.Filter.date(start_date,end_date);#Filter image based on the time frame(start_date and end_date
                try:
                    sentinel_1= ee.ImageCollection('COPERNICUS/S1_GRD');
                    sCollection=sentinel_1\
                    .filterBounds(boundaries) \
                    .filter(season) \
                    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
                    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
                    .filter(ee.Filter.eq('instrumentMode', 'IW'))
              
                    desc=sCollection.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'));
                    asc=sCollection.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'));

                    global composite
                    composite = ee.Image.cat([
                asc.select('VH').mean(),
                asc.select('VV').mean(),
                desc.select('VH').mean()
                ]).focal_median();
   
 #-------------SENTINEL_2A DATA----------------------#  
                    global sentinel_2A
                    sentinel_2A = ee.ImageCollection('COPERNICUS/S2')\
               .filterBounds(boundaries)\
               .filterDate("2020-01-01","2020-03-31")\
               .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",10))\
               .median()\
               .select('B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8', 'B10', 'B11')\
               .clip(boundaries)
                except Exception as e:
                    print("***Hint this may be caused by missing ROI.....please draw your ROI first***")
#                

                sentinel_2Avispar={"min":0, "max":2500,"bands": ['B4','B3','B2']}#Visualization parameters used.

                map_layout.addLayer(sentinel_2A,sentinel_2Avispar, 'Sentinel-2')#Add sentinel_2A to the layer 
        
            except Exception as e:
                print('Sorry an error occurred...please try again')
            else:
                print("Image preprocessing attained successfully")
        
Image_Preprocessing.on_click(preprocess_img_clicked)


In [13]:
def load_model_clicked(b):
    
    with output_widget:
        output_widget.clear_output()
#         map_layout.default_style = {'cursor': 'wait'}
        try:
            Fused_images = sentinel_2A.addBands(composite)#Creation of composite image of sentinel 1 and sentinel 2A
            feature_collection88=gdp.read_file("POINT.shp")#Reading Traing Data
            fused_vis = {
        'min':0,
        'max'  : 2500,
        'bands' : ['B2', 'B3','B4']#Band selection.
           }
            
            feature_collection = geemap.gdf_to_ee(feature_collection88)#FROM GDF TO ee feature collection
            style = {'color': '131313', 'fillColor':' FD0505'}
            map_layout.addLayer(feature_collection.style(**style),{}, "Classification Training ponts")
#             buffer=feature_collection

#Create a buffer:
#             training = point_data.map(lambda f: f.buffer(5))
            training=feature_collection
            label = 'cropland'#Unique property(landcover) with different unique values..
            bands=["B4","B3","B2","VV","VH"]
            global Input
            Input=Fused_images.select(bands)
            trainImage=Input.sampleRegions(**{
    'collection':training,
    'properties':[label],
    'scale':30
})

            trainingData=trainImage.randomColumn()

            trainSet=trainingData.filter(ee.Filter.lessThan('random',0.75))
            testdata=trainingData.filter(ee.Filter.greaterThanOrEquals('random',0.75))


#visualization parameters applied on the Random forest classifier

            init_params = {"numberOfTrees":10, # the number of individual decision tree models
              "variablesPerSplit":None,  # the number of features to use per split
              "minLeafPopulation":1, # smallest sample size possible per leaf
              "bagFraction":0.5, # fraction of data to include for each individual tree model
              "maxNodes":None, # max number of leafs/nodes per tree
               "seed":0}  # random seed for "random" choices like sampling. Setting this allows others to reproduce your exact results even with stocastic parameters
            global classifier

            classifier = ee.Classifier.smileRandomForest(**init_params).train(trainImage, label, bands)
# Classify the image.

#Application of the random forest classifier for the purpose of image classification.
            global classified
            classified=Input.classify(classifier)
            palette = [
  '#008000 ',#Cropland  
  'f6ff0b', #Trees
 ];
            map_layout.addLayer(classified,
            {'min': 0, 'max': 1, 'palette': palette},
             'classification')


            legend_dict = {
 
    'Cropland': '008000 ',#
    'Trees & Roads': 'f6ff0b',
  
    }
            map_layout.add_legend(legend_title="Classification", legend_dict=legend_dict, position='topleft',margin='0 0 10px 10px')
#              output_widget.clear_output()
              #User Classification accuracy using output widget output to map
            print('Computing classification accuracies...')
             
            global train_accuracy
            train_accuracy = classifier.confusionMatrix()
            overall_accuracy= train_accuracy.accuracy()#Overall classification accuracy
            overall_accuracy_perc=overall_accuracy.multiply(100)
            overall_accuracy_tranc=ee.Number(overall_accuracy_perc).format('%.3f')
            print('Overall Accuracy:',overall_accuracy_tranc.getInfo())
            
            kappa_accuracy=train_accuracy.kappa().multiply(100)#Kappa accuracy
            kappa_accuracy_tran=ee.Number(kappa_accuracy).format('%.3f')
            print('Kappa Accuracy:',kappa_accuracy_tran.getInfo())

            
        except Exception as e:
                print("**Hint 1 no training points were found...Please load your training points** \n **Hint 2 no satellite images were found....please carry out image pre-processing** \n oops an Error occurred.....please review the previous step!!!")
        else:
            print("Classification Done!!!")
            
load_model.on_click(load_model_clicked)

In [14]:
#Trial to avoid printing error in the code but a message to the user
def AOI_clicked(b):
        with output_widget:
            output_widget.clear_output()
           
            try:
                global Crop_Land_Mask
                Crop_Land_Mask = classified.updateMask(classified.eq(0))
                biomass_viz=['FF0000']
                map_layout.addLayer(Crop_Land_Mask,{min:0, max:1,'palette':biomass_viz},"Crop Land Mask")
                
            except Exception as e:
                print('An error occurred...Revisit the previous Steps...')
            else:
                print("Crop Land Masking attained successfully")
                output_widget.clear_output()
Update_Mask.on_click(AOI_clicked)

In [15]:
#Trial to avoid printing error in the code but a message to the user
def AOI_clicked(b):
        with output_widget:
            output_widget.clear_output()
           
            try:
                map_layout.remove_legends() 
                map_layout.remove_colorbars()
                global NDVI
                NDVI = sentinel_2A.normalizedDifference(['B8', 'B4'])#normalized difference is computed as (first − second) / (first + second).
                ndvivis_parametres = {'min':0, 'max':1, 'palette': ['red','brown','yellow', 'green'] }#NDVI visualization parameters
                map_layout.addLayer(NDVI, ndvivis_parametres, 'NDVI(Normalized Difference Vegetation Index)')#Add Normalized Difference Vegetation Index to the layers
                vis_params = {
                    'min': 0,
                    'max': 1,
                    'palette':['red','brown','yellow', 'green'],
                }
                colors = vis_params['palette']
                vmin = vis_params['min']
                vmax = vis_params['max']
                map_layout.add_colorbar(vis_params,label='NDVI Analysis')
                
                map_layout.addLayerControl()
#                 global legend_dict
                legend_dict = {
                       'Non-crops(0 to 0.18)': 'FF0000',
                       'Unhealthly crops(0.18 to 0.41)': 'A52A2A',
                       'Moderately healthy crops(0.41 to 0.0.69)': 'FFFF00',
                       'Very healthy crops(0.69 to 1.0)': '008000',}
                map_layout.add_legend(legend_title="(Crop Health)NDVI", legend_dict=legend_dict)
                
            except Exception as e:
                print('Please run the image pre-processing first!!!!')
            else:
                print("Your NDVI was there successfully")
                output_widget.clear_output()
NDVI_execute.on_click(AOI_clicked)

In [16]:
#Creation of an output widgets to ouput outside the map.
user_out = widgets.Output(layout={'border': '1px solid green'})
user_out

Output(layout=Layout(border='1px solid green'))

In [17]:

#Trial to avoid printing error in the code but a message to the user
def AOI_clicked(b):
        with user_out:
#             output_widget.clear_output()
           
            try:
        
                  
#Extract Gross primary Product(GPP)
                out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
                out_gppCollection_stats = os.path.join(out_dir, 'GPP_Mean_Value.csv')
                
                if not os.path.exists(out_dir):
                    os.makedirs(out_dir)

                    
# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
                geemap.zonal_statistics(gppCollection, boundaries, out_gppCollection_stats,statistics_type='MEAN', scale=10)
        
#Extract NDVI
                out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
                NDVI_stats = os.path.join(out_dir, 'NDVI_Mean_Value.csv')
                
                if not os.path.exists(out_dir):
                    os.makedirs(out_dir)

                    
# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
                geemap.zonal_statistics(NDVI, boundaries, NDVI_stats,statistics_type='MEAN', scale=10)
                


               
            except Exception as e:
                print('Please their is a chance that you may skip a step...')
            else:
                print("Successfully run the model....")
                
Export_data.on_click(AOI_clicked)

In [18]:
#Trial to avoid printing error in the code but a message to the user
def predict_clicked(b):
        with output_widget:
            output_widget.clear_output()
           
            try:
                
                global Biomass_df
                Biomass_df=pd.read_csv("ee-Biomass_Training_Data.csv")# define data
                
                Biomass_df['Biomass_df_mean']=Biomass_df.mean(axis=1)
                
                GPP_df=pd.read_csv("ee-GPP_Training_Data.csv")# define data
                
                GPP_df['GPP_df_mean']=GPP_df.mean(axis=1)
                
                Biomass_df_Drop=Biomass_df.drop(columns=["system:time_start","point_0","point_01","point_02","point_03","point_04","point_05","point_06","point_07","point_08","point_09","point_010","point_011","point_012","point_013","BM_point_014"])
                
                GPP_df_Drop=GPP_df.drop(columns=["system:time_start","point_0","point_01","point_02","point_03","point_04","point_05","point_06","point_07","point_08","point_09","point_010","point_011","point_012","point_013","GPP_point_014"])
                
                
                Df_list=[GPP_df_Drop,Biomass_df_Drop]
                
                General_Train_df=pd.concat(Df_list,axis=1)
                #Assigning the selected rows axis eg x and y axis.
#Define x and Y
                x_Biomass=General_Train_df.iloc[:,:1].values
                y_Biomass=General_Train_df.iloc[:,1:2].values #Select Y and multiply by 100 i.e exponential
                
#80% training and 20% validation
#Aspect of Random With random_state=None , we get different train and test sets across different executions and the shuffling
# process is out of control. With random_state=0 , we get the same train and test sets across different executions.
                from sklearn.model_selection import train_test_split
                x_train_biomass, x_test_biomass,y_train_biomass, y_test_biomass= train_test_split(x_Biomass,y_Biomass,test_size=0.2,random_state=0)



                from sklearn.linear_model import LinearRegression
# statistical way of measuring the relationship between one or more independent variables vs one dependent variable. 
                regressor = LinearRegression()
                regressor.fit(x_train_biomass,y_train_biomass)
                y_pred_biomass_carbon= regressor.predict(x_test_biomass)
############################################
#Reading of the extracted Data from the Local Disk
#Change To your downloads default directory.....!!!!!!!!!!!!!! 
############################################
                GPP_extracted_values=pd.read_csv(r"C:\Users\user\Downloads\GPP_Mean_Value.csv")
#Meaning
                GPP_extracted_values['GPP Mean Value']=GPP_extracted_values.mean(axis=1)
#Selection of a single column By name
                Gpp_Input_Value=GPP_extracted_values[["GPP Mean Value"]]
                global GPP_Mean
                GPP_Mean=GPP_extracted_values[["GPP Mean Value"]]
            
############################################
#Reading of the extracted NDVI from the Local Disk
#Change To your downloads default directory.....!!!!!!!!!!!!!! 
############################################
                NDVI_extracted_values=pd.read_csv(r"C:\Users\user\Downloads\NDVI_Mean_Value.csv")
#Mean
                NDVI_extracted_values['NDVI Mean value']=NDVI_extracted_values.mean(axis=1)
#Selection of a single column By name
                NDVI_Input_Value=NDVI_extracted_values[["NDVI Mean value"]]
    
                predicted_Biomass=regressor.predict(Gpp_Input_Value)

#Conversion of a 2D to 3D----Reshaping
                x=predicted_Biomass[0]
                predicted_value=x[0]
# Round off to-4 figure
                predicted_biomass_value=(round(predicted_value, 4))
                print(GPP_Mean)
                print(NDVI_Input_Value)
                print("Predicted Biomass will be:",predicted_biomass_value)

     
            except Exception as e:
                print('Please Re-Try Again!!!.....')
            else:
                print("Successfully run the selected....")   
Data_Input.on_click(predict_clicked)

In [19]:
# Click event handler for Image processing process e.g sentinel data

def submit_clicked(b):
     with output_widget:
            try:
                output_widget.clear_output()
                map_layout.remove_legends()
                start_date="2018-01-01"#Set start_date(yy/mon/day)
                end_date="2022-03-31"#Set End_date(yy/mon/day)
            
                global region # Set the region of interest by simply drawing a polygon on the map
                region = boundaries
                if region is None:
                    region = ee.Geometry.BBox(-61.18116254022864,6.614609265030635, -61.18116254022864,6.769074870340775)

                image = geemap.dynamic_world_s2(region, start_date, end_date)
                vis_params3 = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
                global landcover
                # Create Dynamic World land cover composite
                landcover = geemap.dynamic_world(region, start_date, end_date, return_type='hillshade')

                # Add legend to the map
                map_layout.addLayer(landcover.clip(boundaries),{}, 'LULCC') 
                map_layout.add_legend(title="Land Cover", builtin_legend='Dynamic_World')
                

            except Exception as e:
                print('Sorry an error occurred...please try again')
            else:
                print("LULC attained successfully")
        
LULCC.on_click(submit_clicked)

In [20]:
def reset_me(b):
    with output_widget:
        output_widget.clear_output()
        try:
            map_layout.remove_legends() 
            map_layout.remove_colorbar()
            map_layout.remove_drawn_features()
            map_layout.remove_ee_layer('Dynamic_World')
            map_layout.remove_ee_layer("Classification Training ponts")
            map_layout.remove_ee_layer("Region of Intrest")
            map_layout.remove_ee_layer('Sentinel-2')
            map_layout.remove_ee_layer('classification')
            map_layout.remove_ee_layer('Crop Land Mask')
            map_layout.remove_ee_layer('S5P CO')
            map_layout.remove_ee_layer('Gross Primary Product')
            map_layout.remove_ee_layer('boundaries')
        except Exception as e:
            print("Sorry an error just occures")
        else:
            print("reset successfully....")

Reset_button.on_click(reset_me)